In [7]:
from dijkstra import Graph
import ipywidgets as widgets
from IPython.display import display, HTML
import time

class Node(widgets.Button):
    def __init__(self, *args, **kwargs):
        super(Node, self).__init__(*args, **kwargs)
        self.name = kwargs["name"]
        self.type = kwargs.get("type", "default")

class App(widgets.VBox):
    def __init__(self, *args, **kwargs):
        self.rows = kwargs.get("rows", 9)
        self.cols = kwargs.get("cols", 16)
        custom_css = """
        <style>
        .my-button {
            border-radius: 12px;
            background-color: #1E90FF;
            color: black;
            border: none;
            padding: 5px 10px;
            text-align: center;
            font-size: 11px;
            width: 52px;
            height: 52px;
        }
        </style>
        """
        
        self.nodes = []
        for i in range(self.rows):
            cols = []
            for j in range(self.cols):
                btn = Node(name=(i, j))
                btn.add_class("my-button")
                btn.on_click(self.on_btn_click)
                cols.append(btn)
            self.nodes.append(cols)

        button_css = """
        <style>
        .shiny-button {
            background-color: #ff7f50;
            border: none;
            color: black;
            padding: 15px 32px;
            text-align: center;
            text-decoration: none;
            display: inline-block;
            font-size: 16px;
            margin: 4px 2px;
            cursor: pointer;
            border-radius: 12px;
            box-shadow: 0 9px #999;
            transition-duration: 0.4s;
        }
        .shiny-button:hover {
            background-color: #ff6347;
        }
        .shiny-button:active {
            background-color: #ff6347;
            box-shadow: 0 5px #666;
            transform: translateY(4px);
        }
        </style>
        """

        self.clear_path_btn = widgets.Button(description="Clear Path")
        self.clear_path_btn.style.button_color = "LightYellow"
        self.clear_path_btn.layout.width = "150px"
        self.clear_path_btn.layout.height = "42px"
        self.clear_path_btn.add_class('shiny-button')
        self.clear_path_btn.on_click(self.on_clear_path_btn_click)

        self.clear_board_btn = widgets.Button(description="Clear")
        self.clear_board_btn.style.button_color = "LightCoral"
        self.clear_board_btn.layout.width = "150px"
        self.clear_board_btn.layout.height = "42px"
        self.clear_board_btn.add_class('shiny-button')
        self.clear_board_btn.on_click(self.on_clear_board_btn_click)

        self.go_btn = widgets.Button(description="Go!")
        self.go_btn.style.button_color = "DarkSeaGreen"
        self.go_btn.layout.width = "150px"
        self.go_btn.layout.height = "42px"
        self.go_btn.add_class('shiny-button')
        self.go_btn.on_click(self.on_go_btn_click)

        self.output = widgets.Output()

        self.about = widgets.HTML(
            value="""
            <div style="text-align: center; font-size: 14px; color: #444; padding-top: 10px; max-width: 800px; margin: 0 auto;">
                <p id="info" style="font-size: larger;">
                    A website for ambulance personnel to effectively find a route to the areas in need
                </p>
                <p id="info" style="font-size: larger;">
                    According to the National Institutes of Health, 700 people died due to poor ambulance response time in rural areas, just in Ireland
                </p>
                <p id="info" style="font-size: larger;">
                    If websites like this can even save one person, it's worth it
                </p>
            </div>
            """
        )

        super(App, self).__init__(
            children=[
                widgets.HTML(
                    value="""
                    <div style="text-align: center; font-size: 28px; font-weight: bold; color: gray; margin-bottom: 10px;">
                        Ambulance Pathfinder
                    </div>
                    """
                ),
                widgets.VBox([widgets.HBox(node) for node in self.nodes], layout=widgets.Layout(align_items='center')),
                widgets.HBox([self.clear_path_btn, self.go_btn, self.clear_board_btn], layout=widgets.Layout(justify_content='space-around', padding='20px')),
                self.output,
                self.about
            ],
        )
        
        display(HTML(custom_css))
        display(HTML(button_css))
        
    def show_error_message(self, message):
        with self.output:
            self.output.clear_output()
            error_message = widgets.HTML(
                value=f'<div style="text-align: center; color: red; font-size: 20px;">Error</div>'
                      f'<div style="text-align: center; color: gray;">{message}</div>'
            )
            ok_button = widgets.Button(
                description='OK',
                button_style='info',
                layout=widgets.Layout(width='100px', margin='0 auto', display='block')
            )
            ok_button.on_click(self.clear_error_message)
            display(widgets.VBox([error_message, ok_button]))

    def clear_error_message(self, b=None):
        with self.output:
            self.output.clear_output()

    def on_clear_path_btn_click(self, *args):
        btns = []
        for i in range(self.rows):
            for j in range(self.cols):
                btn = self.nodes[i][j]
                if btn.style.button_color == "LightGray":
                    btns.append(btn)
                    
        for btn in btns[::-1]:
            btn.style.button_color = "DodgerBlue"
            btn.type = "default"
            btn.description = " "
            btn.icon = " "
            time.sleep(0.02)

    def on_clear_board_btn_click(self, *args):
        for i in range(self.rows):
            for j in range(self.cols):
                btn = self.nodes[i][j]
                btn.style.button_color = "DodgerBlue"
                btn.type = "default"
                btn.description = " "
                btn.icon = " "

    def on_btn_click(self, node):
        if node.type == "default":
            node.type = "start"
            node.style.font_size = '11px'
            node.description = "Start"
            node.style.button_color = "MediumSeaGreen"
        elif node.type == "start":
            node.type = "block"
            node.description = " "
            node.style.font_size = '20px'
            node.icon = "tree"
            node.style.button_color = "OrangeRed"
        elif node.type == "block":
            node.type = "end"
            node.style.font_size = '11px'
            node.icon = " "
            node.description = "End"
            node.style.button_color = "White"
        else:
            node.type = "default"
            node.style.font_size = '11px'
            node.description = " "
            node.icon = " "
            node.style.button_color = "DodgerBlue"

    def on_go_btn_click(self, *args):
        tups = []
        points = [(-1, 0), (0, 1), (1, 0), (0, -1)]
        start = None
        end = None

        for i in range(self.rows):
            for j in range(self.cols):
                btn = self.nodes[i][j]

                for dx, dy in points:
                    x, y = i + dx, j + dy
                    if 0 <= x < self.rows and 0 <= y < self.cols:
                        if btn.type == "start":
                            start = (i, j)
                        if btn.type == "end":
                            end = (i, j)
                        
                        if btn.type == "block":
                            tups.append(((i, j), (x, y), float("inf")))
                        else:
                            tups.append(((i, j), (x, y), 1))

        if not start or not end:
            self.show_error_message("Please select both start and end nodes to proceed.")
            return

        graph = Graph(tups)
        shortest_path = graph.dijkstra(start, end)

        if not shortest_path:
            self.show_error_message("No path found.")
            return

        self.animate_path(shortest_path[0])

    def animate_path(self, path):
        for i in range(len(path)):
            y, x = path[i]
            btn = self.nodes[y][x]
            if i == 0:
                btn.style.button_color = "MediumSeaGreen"
            elif i == len(path) - 1:
                btn.style.button_color = "White"
            else:
                btn.style.button_color = "LightGray"
                time.sleep(0.065)

            if i != 0 and i != len(path) - 1:
                a, b = path[i + 1]
                btn.descripton = " "
                if b == x + 1:
                    btn.icon = "arrow-right"
                elif b == x - 1:
                    btn.icon = "arrow-left"
                elif a == y + 1:
                    btn.icon = "arrow-down"
                elif a == y - 1:
                    btn.icon = "arrow-up"


In [8]:
App()

App(children=(HTML(value='\n                    <div style="text-align: center; font-size: 28px; font-weight: …